In [1]:
# install packages
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from osgeo import gdal
from osgeo import ogr
from osgeo import gdalconst

In [2]:
import os
os.getcwd()

'c:\\Users\\admin\\Desktop\\GitHub\\riskaudit\\notebooks'

In [ ]:
shp_fn =  r"C:\Users\admin\Desktop\GitHub\riskaudit\data\groundtruth\PHL_QC_EMI_AUGUST2022\barangay_boundary\NAME_V2_North Fair.gpkg"


# open shapefile and corresponding raster
data = gdal.Open(sar_fn, gdalconst.GA_ReadOnly)
dataArray = data.ReadAsArray()

# hardcode output coordinate ref system
epsgOut = 4326

# Get coordinate information for input raster
# x_min/x_max, y_min/y_max correspond to bounding box of image
# x and y res are the verticle and horizontal spatial resolutions
geo_transform = data.GetGeoTransform()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * data.RasterXSize
y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize
pixel_width = geo_transform[1]

# Open shapefile in geopandas
dataShp = geopandas.read_file(shp_fn)
# change CRS from polygon to raster file
dataShp = dataShp.to_crs(epsg=epsgOut)
# convert geopandas into ogr Datasource
shp_temp = ogr.Open(dataShp.to_json())
# extract layer from ogr Datasource
mb_l = shp_temp.GetLayer()

# create dataset as geotiff
target_ds = gdal.GetDriverByName('GTiff').Create(
    output, x_res, y_res, 1, gdal.GDT_Byte)
# attach geo information to dataset, e.g. resolution and projection
target_ds.SetGeoTransform(
    (x_min, pixel_width, 0, y_min, 0, pixel_width))
band = target_ds.GetRasterBand(1)
NoData_value = 100
band.SetNoDataValue(NoData_value)
band.FlushCache()
# Need to change the value after 'ATTRIBUTE=". This corresponds to the field (column)
# in the shapefile's attribute table in QGIS that you want to use.
gdal.RasterizeLayer(target_ds, [1], mb_l, options=["ATTRIBUTE=CT"])

# important to do this to prevent errors coming up
target_ds = None


In [20]:
# read barangay extent
brgy_path = r"C:\Users\admin\Desktop\GitHub\riskaudit\data\groundtruth\PHL_QC_EMI_AUGUST2022\barangay_boundary\NAME_V2_North Fair.gpkg"
brgy = gpd.read_file(brgy_path)
